In [1]:
import kaggle
import pandas as pd

# Authenticate with the Kaggle API
kaggle.api.authenticate()

# Download the dataset
kaggle.api.dataset_download_files('lokeshparab/amazon-products-dataset', path='.', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/lokeshparab/amazon-products-dataset


In [2]:
import os

# Obtenir le chemin actuel du notebook
chemin_actuel = os.getcwd()
print("Le chemin actuel du notebook est :", chemin_actuel)

Le chemin actuel du notebook est : c:\Users\thiba\GitHub\mistral_retail\data


In [3]:
# Chemin vers le dossier contenant les fichiers CSV
dossier_csv = chemin_actuel

# Liste pour stocker les DataFrames individuels
dataframes = []

# Parcourir tous les fichiers dans le dossier
for fichier in os.listdir(dossier_csv):
    if fichier.endswith('.csv'):
        chemin_fichier = os.path.join(dossier_csv, fichier)
        df = pd.read_csv(chemin_fichier)
        dataframes.append(df)

# Concaténer tous les DataFrames en un seul
amazon_dataset = pd.concat(dataframes, ignore_index=True)

# Afficher les premières lignes du DataFrame complet
print(amazon_dataset.head())

                                                name main_category  \
0  Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...    appliances   
1  LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...    appliances   
2  LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...    appliances   
3  LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...    appliances   
4  Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...    appliances   

       sub_category                                              image  \
0  Air Conditioners  https://m.media-amazon.com/images/I/31UISB90sY...   
1  Air Conditioners  https://m.media-amazon.com/images/I/51JFb7FctD...   
2  Air Conditioners  https://m.media-amazon.com/images/I/51JFb7FctD...   
3  Air Conditioners  https://m.media-amazon.com/images/I/51JFb7FctD...   
4  Air Conditioners  https://m.media-amazon.com/images/I/41lrtqXPiW...   

                                                link ratings no_of_ratings  \
0  https://www.amazon.in/Lloyd-Inverter-Convertib...    

In [4]:
amazon_dataset.drop(["link", "image", "link", "Unnamed: 0"], axis=1, inplace=True)

In [5]:
categories_to_remove = ['Washing Machines', "Refrigerators", "Air Conditioners",
                        "Janitorial and Sanitation Supplies", "Industrial and Scientific Supplies",
                        "Indoor Lighting", "Home Storage", "Household Supplies", "Diapers", "Car Parts"]

# Supprimer les lignes où la colonne 'Category' est dans la liste
gifts_dataset = amazon_dataset[~amazon_dataset['sub_category'].isin(categories_to_remove)]

In [6]:
gifts_dataset.isna().any()

name              False
main_category     False
sub_category      False
ratings            True
no_of_ratings      True
discount_price     True
actual_price       True
dtype: bool

In [7]:
gifts_dataset = gifts_dataset.dropna(subset=["actual_price"])

In [8]:
gifts_dataset = gifts_dataset.drop_duplicates(subset=None, keep='first', ignore_index=False)

In [9]:
gifts_dataset['discount_price'] = gifts_dataset['discount_price'].str.replace('₹', '').str.replace(',', '').astype(float)
gifts_dataset["actual_price"] = gifts_dataset["actual_price"].str.replace('₹', '').str.replace(',', '').astype(float)

In [10]:
gifts_dataset['ratings'] = pd.to_numeric(gifts_dataset['ratings'], errors='coerce')

gifts_dataset['no_of_ratings'] = pd.to_numeric(gifts_dataset['no_of_ratings'], errors='coerce')

gifts_dataset = gifts_dataset.dropna(subset=['ratings', 'no_of_ratings'])
gifts_dataset['no_of_ratings'] = gifts_dataset['no_of_ratings'].astype(int)


In [11]:
gifts_dataset['discount_price'] = gifts_dataset['discount_price'].fillna(gifts_dataset["actual_price"])

In [ ]:
gifts_dataset[['discount_price', "actual_price"]] = gifts_dataset[['discount_price', "actual_price"]]/88

In [12]:
gifts_dataset_encoded = pd.get_dummies(gifts_dataset, columns=["main_category", "sub_category"], dummy_na=True, drop_first=True, prefix_sep=' ', prefix='')

In [13]:
column_sums = gifts_dataset_encoded.iloc[:, 1:].sum()

columns_to_drop = column_sums[column_sums < 1].index

gifts_dataset_encoded = gifts_dataset_encoded.drop(columns=columns_to_drop)

In [14]:
gifts_dataset_encoded

,name,ratings,no_of_ratings,discount_price,actual_price,appliances,bags & luggage,beauty & health,car & motorbike,grocery & gourmet foods,...,Toys & Games,Toys Gifting Store,Travel Accessories,Travel Duffles,Value Bazaar,Wallets,Watches,Western Wear,Women's Fashion,Yoga
722,Glun Multipurpose Portable Electronic Digital ...,3.8,365,199.00,899.00,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
738,"Pigeon Healthifry Digital Air Fryer, 360° High...",3.9,692,3529.00,7995.00,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
748,Farberware Portable Electric USB Juice Maker J...,2.6,164,489.00,1199.00,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
751,Tosaa 18 Pcs - 3 Different Size Plastic Food S...,3.9,82,59.00,99.00,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
775,Venus Electronic Digital Kitchen Weighing Scal...,3.8,667,299.00,1599.00,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054186,Lucky Brand Boys' Heritage Short,4.8,7,4497.75,4497.75,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1055118,Kids Mandi Brainvita Board Game with 32 Marble...,3.9,315,189.00,239.00,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1058101,Surf Excel Matic Top Load Liquid Detergent 3.2...,4.5,450,550.00,700.00,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
1058122,Surf Excel Matic Front Load Liquid Detergent 3...,4.5,313,1299.00,1499.00,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
